In [11]:
# parameters cell
input_dir_pdbqt = None
output_dir_sdf = None
pdb_file = ''
ligands_file = ''
ligands_files_dir = None
reference_file = ''
write_scores = True

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import sys, os, glob
sys.path.append('/code/tools')
from scoring.plif import PlifScore
from misc import vina_pdbqt_to_sdf, is_interactive
from ui.file_widgets import DockButtons, PathSelector, InputBoxSet
import scrapbook as sb

# Scoring vina results: PLIF (protein-ligand interaction fingerprint) against reference

---

### 1. Convert pdbqt files to sdf files 

---

input directory = the place where all of the pdbqt files from a docking run are  
output directory = where to write the sdfs out to 

In [5]:
if is_interactive():
    iconv = InputBoxSet(tlabel='input dir:')
    display(iconv.render())


A Jupyter Widget

In [6]:
if is_interactive():
    oconv = InputBoxSet(tlabel='output dir:')
    display(oconv.render())

A Jupyter Widget

In [7]:
if is_interactive():
    input_dir_pdbqt = iconv.inbox.value
    output_dir_sdf = oconv.inbox.value

infiles = glob.glob(os.path.join(input_dir_pdbqt, '*.pdbqt'))

if not os.path.isdir(output_dir_sdf):
    os.makedirs(output_dir_sdf)

[vina_pdbqt_to_sdf(ifile=i, 
                   output=os.path.join(output_dir_sdf, i.split('/')[-1].replace('.pdbqt', '.sdf'))) 
 for i in infiles]

---
### 2. Select some input files for the PLIF scoring
---

#### 2.1.  Select the protein and reference ligand: 
*(N.B. run the next 2 cells to show the file dialogues)*
1. click on the arrow in the dropdown box at the top
2. select the relevant file in the dialogue 
3. click the button next to either protein, query ligands or reference to set the file

---

#### 2.2.  Select the query molecules to score against the reference ligand:

**There are two options here:** 

1. Use one file to specify which ligands to score: use the buttons as described above
2. Specify a directory of sdf files of multiple files of ligands to score: type the directory to search in the 'multiple dir' input box, and click 'set'


In [8]:
if is_interactive():
    p = PathSelector(start_dir=os.getcwd())
    display(p.accord)

    b = DockButtons(path_selector=p)
    display(b.render_buttons())

A Jupyter Widget

A Jupyter Widget

In [9]:
if is_interactive():
    pdb_file = b.pdb_button.description
    ligands_file = b.lig_button.description
    ref_file = b.ref_button.description

In [10]:
if is_interactive():
    inp = InputBoxSet(tlabel='multiple dir:')
    display(inp.render())

A Jupyter Widget

---
### 3. run the PLIF scoring
---

This returns: 

**For one file:** a dataframe with the scores in  
**For multiple files:** a list of dataframes with the scores in

You can also specify not to write the scores back out to the input sdf files by changing ```write_scores=True``` to ```write_scores=False``` below

In [12]:
if is_interactive():
    ligands_files_dir = inp.inbox.value
    write_scores = True

if os.path.isdir(ligands_files_dir):
    files = glob.glob(os.path.join(ligands_files_dir, '*.sdf'))
        
else:
    files = [ligands_file]
    
plif = PlifScore(protein_pdb=pdb_file, 
                 reference_file=reference_file)

if len(files)==1:
    df = plif.score_conformers(file=ligands_file, write=write_scores, score_col='PLIF_SCORE')
    
else:
    from joblib import Parallel, delayed
    import multiprocessing
    from tqdm import tqdm
    import pandas as pd
    
    num_cores = multiprocessing.cpu_count()
    df_list = Parallel(n_jobs=num_cores)(delayed(plif.score_conformers)(
        file=f, write=write_scores, score_col='PLIF_SCORE'
    ) for f in tqdm(files))

ValueError:  files are not supported for the protein.